<a href="https://colab.research.google.com/github/Muppidathe/deep_learning/blob/main/ann_tensorflow(regression).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder,StandardScaler,OneHotEncoder,MinMaxScaler
import pickle
import numpy as np

In [2]:
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset/Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


# data transformation


In [4]:
#drop unwanted column
df=df.drop(columns=['RowNumber','CustomerId','Surname'])
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
print(df['Gender'].unique())
gender_encoder=LabelEncoder()
df['Gender']=gender_encoder.fit_transform(df['Gender'])
df.head()

['Female' 'Male']


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [6]:
print(df['Geography'].unique())
print('len: ',len(df['Geography'].unique()))
geo_encoder=OneHotEncoder()
temp_df=geo_encoder.fit_transform(df[['Geography']]).toarray()
temp_df=pd.DataFrame(temp_df,columns=geo_encoder.get_feature_names_out())
temp_df.head()

['France' 'Spain' 'Germany']
len:  3


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [7]:
x=pd.concat([temp_df,df.drop(columns=['Geography','EstimatedSalary'])],axis=1)
y=df['EstimatedSalary']
x.shape

(10000, 12)

# train test spliting

In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.2,random_state=42)
x_train.shape,y_train.shape

((8000, 12), (8000,))

#  scaling all the feature using minmaxscaler

In [11]:
scaler=MinMaxScaler()
x_transformed=scaler.fit_transform(x_train)
x_transformed[:2,:]

array([[1.        , 0.        , 0.        , 0.672     , 1.        ,
        0.18918919, 0.6       , 0.        , 0.33333333, 1.        ,
        1.        , 0.        ],
       [0.        , 1.        , 0.        , 0.564     , 1.        ,
        0.32432432, 0.4       , 0.47678561, 0.33333333, 1.        ,
        1.        , 0.        ]])

# designing model and compiling

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense,Input
input=Input(shape=(x_transformed.shape[1],))
hidden=Dense(64,activation=tf.keras.activations.relu)(input)
hidden=Dense(32,activation=tf.keras.activations.relu)(hidden)
output=Dense(1)(hidden)
model=Model(inputs=input,outputs=output)
model.compile(loss=tf.keras.losses.MeanSquaredError,optimizer=tf.keras.optimizers.Adam(learning_rate=.01,beta_1=.9,beta_2=.99),metrics=[tf.keras.metrics.R2Score,tf.keras.metrics.MeanAbsoluteError])

# training model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
earlystopping=EarlyStopping(monitor='val_loss',patience=80,restore_best_weights=True)
model.fit(x_train,y_train,validation_data=(x_test,y_test),batch_size=1024,epochs=100,callbacks=[earlystopping])

Epoch 1/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 10532295680.0000 - mean_absolute_error: 86239.4219 - r2_score: -2.1935 - val_loss: 7883859968.0000 - val_mean_absolute_error: 72890.1094 - val_r2_score: -1.3881
Epoch 2/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 7664886784.0000 - mean_absolute_error: 71611.8047 - r2_score: -1.3149 - val_loss: 7032441856.0000 - val_mean_absolute_error: 68475.3672 - val_r2_score: -1.1302
Epoch 3/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 7304466944.0000 - mean_absolute_error: 69780.2188 - r2_score: -1.2086 - val_loss: 6807768576.0000 - val_mean_absolute_error: 67524.2188 - val_r2_score: -1.0622
Epoch 4/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 7105491456.0000 - mean_absolute_error: 68548.8906 - r2_score: -1.1702 - val_loss: 6703899648.0000 - val_mean_absolute_error: 67045.1250 - val_r2_score: -1.0307
Epoch 5/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 6932939776.0000 - mean_absolute_error: 67800.0312 - r2_score: -1.1328 -

In [ ]:
model.predict(x_test)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


array([[ 95566.73],
       [ 94029.27],
       [ 96705.62],
       ...,
       [101620.33],
       [ 93681.64],
       [ 93032.64]], dtype=float32)